In [1]:
import json
import pandas as pd
from pathlib import Path
from pandas import json_normalize
from tqdm import tqdm

In [2]:
assessment_data = Path(f'out/assessment_data')
assessment_data.exists()

True

In [3]:
assessment_dir_paths = list(assessment_data.glob('*/'))
assessment_dir_paths

[PosixPath('out/assessment_data/100-trials_conf-geq-70_sample-from-dswx')]

In [4]:
def get_dswx_ids(assessment_dir) -> list:
    dswx_verification_paths = list(assessment_dir.glob('*/'))
    dswx_ids = [path.name for path in dswx_verification_paths]
    # Remove DSWx paths
    dswx_ids = list(filter(lambda dswx_id: '.' != dswx_id[0], dswx_ids))
    return dswx_ids


def combine_requirement_verification_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    out_dir.mkdir(exist_ok=True, parents=True)
    
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    def read_one_requirements_json(dswx_id):
        path = assessment_dir_path / dswx_id / f'requirement_verification_{dswx_id}.json'
        data = json.load(open(path))
        return data
    
    data = list(map(read_one_requirements_json, dswx_ids))
    df = pd.DataFrame(data)
    
    cols = ['surface_water','partial_surface_water']
    df[cols + ['dswx_id']].groupby(cols).count()
    
    out_path = out_dir / 'verification_results.csv'
    df.to_csv(out_path, index=False)
    
    return df, out_path

In [5]:
dfs, out_paths = zip(*list(map(combine_requirement_verification_for_one_assessment, tqdm(assessment_dir_paths))))

100%|█| 1/1 [00:00<00:00, 101.82it/


In [6]:
dfs[0]

,dswx_id,surface_water,partial_surface_water,dswx-WTR,dswx-BWTR,dswx-CONF,dswx-DIAG,dswx-WTR-1,dswx-WTR-2,dswx-LAND,...,hls_url_B04,hls_url_B05,hls_url_B06,hls_url_B07,hls_url_B08,hls_url_B09,hls_url_B10,hls_url_B11,hls_url_B12,hls_url_Fmask
0,OPERA_L3_DSWx_HLS_T04VCM_20210928T220529Z_2022...,True,True,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...


In [7]:
out_paths[0]

PosixPath('out/verification_stats_agg/100-trials_conf-geq-70_sample-from-dswx/verification_results.csv')

# Additional Metrics

In [8]:
def combine_metrics_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    
    def read_one_assessment(dswx_id):
        path = assessment_dir_path / dswx_id / f'stats_{dswx_id}.json'
        data = pd.DataFrame(pd.json_normalize(json.load(open(path))))
        return data
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    data_assessments = list(map(read_one_assessment, dswx_ids))
    df_metrics = pd.concat(data_assessments, axis=0).reset_index(drop=True)
    df_metrics.to_csv(out_dir / 'metrics.csv', index=False)
    out_path = out_dir / 'metrics.csv'
    df_metrics.to_csv(out_path, index=False)
    return df_metrics, out_path

In [9]:
dfs, out_paths = zip(*list(map(combine_metrics_for_one_assessment, tqdm(assessment_dir_paths))))

100%|█| 1/1 [00:00<00:00, 227.31it/


In [10]:
dfs[0]

,total_accuracy.mean,total_accuracy.std,acc_per_class.Not_Water.mean,acc_per_class.Not_Water.std,acc_per_class.Open_Surface_Water.mean,acc_per_class.Open_Surface_Water.std,acc_per_class.Partial_Surface_Water.mean,acc_per_class.Partial_Surface_Water.std,confusion_matrix.Not_Water_OPERA_DSWx.Not_Water_OPERA_Validation.mean,confusion_matrix.Not_Water_OPERA_DSWx.Not_Water_OPERA_Validation.std,...,f1_per_class.Partial_Surface_Water.mean,f1_per_class.Partial_Surface_Water.std,supp_per_class.Not_Water.mean,supp_per_class.Not_Water.std,supp_per_class.Open_Surface_Water.mean,supp_per_class.Open_Surface_Water.std,supp_per_class.Partial_Surface_Water.mean,supp_per_class.Partial_Surface_Water.std,dswx_id,planet_id
0,0.86018,0.012613,0.951886,0.00898,0.907515,0.009448,0.860958,0.012475,160.77,2.469429,...,0.844865,0.014615,167.0,0.0,0.0,0.0,167.0,0.0,OPERA_L3_DSWx_HLS_T04VCM_20210928T220529Z_2022...,20210928_211311_91_2457


In [11]:
out_paths[0]

PosixPath('out/verification_stats_agg/100-trials_conf-geq-70_sample-from-dswx/metrics.csv')